In [93]:
import pandas as pd

In [94]:
df = pd.read_csv("../data/data_transformed.csv")

In [95]:
df.isna().sum()

event-id                        0
location-long                   0
location-lat                    0
argos:lat1                      0
argos:lat2                     85
argos:lon1                      0
argos:lon2                     85
individual-local-identifier     0
date                            0
time                            0
dtype: int64

In [96]:
df = df.dropna()

In [97]:
import datetime as dt

In [98]:
df["year"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").year)


In [99]:
df["month"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").month)

In [100]:
df=df.drop(columns=["event-id", "date", "time", "year", "month"])

In [101]:
for elem in df["individual-local-identifier"].unique():
     df.loc[df["individual-local-identifier"]==elem].to_csv(f"../data/datasets_by_year/{elem}.csv")

In [102]:
df_23043 = pd.read_csv("../data/datasets_by_year/1999CA-Bmu-23043.csv") #individuo con la ruta más larga

In [103]:
df_23043

,Unnamed: 0,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2,individual-local-identifier
0,3173,-120.143,34.120,34.120,35.544,-120.143,-127.027,1999CA-Bmu-23043
1,3174,-120.145,34.147,34.147,32.243,-120.145,-129.529,1999CA-Bmu-23043
2,3175,-119.740,33.980,33.980,38.156,-119.740,-139.994,1999CA-Bmu-23043
3,3176,-119.771,34.163,34.163,28.573,-119.771,-145.945,1999CA-Bmu-23043
4,3177,-119.688,34.350,34.350,42.993,-119.688,-76.494,1999CA-Bmu-23043
...,...,...,...,...,...,...,...,...
410,3583,-122.007,21.730,21.730,22.057,-122.007,-123.562,1999CA-Bmu-23043
411,3584,-122.871,21.247,21.247,20.439,-122.871,-119.302,1999CA-Bmu-23043
412,3585,-122.755,21.083,21.083,16.252,-122.755,-101.167,1999CA-Bmu-23043
413,3586,-122.272,20.831,20.831,14.861,-122.272,-149.760,1999CA-Bmu-23043


In [104]:
import tensorflow as tf
from tensorflow.data import Dataset

In [105]:
df_23043 = df_23043.drop(columns=["Unnamed: 0", "individual-local-identifier"])
#ballena con ruta más larga

In [106]:
df_23043

,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2
0,-120.143,34.120,34.120,35.544,-120.143,-127.027
1,-120.145,34.147,34.147,32.243,-120.145,-129.529
2,-119.740,33.980,33.980,38.156,-119.740,-139.994
3,-119.771,34.163,34.163,28.573,-119.771,-145.945
4,-119.688,34.350,34.350,42.993,-119.688,-76.494
...,...,...,...,...,...,...
410,-122.007,21.730,21.730,22.057,-122.007,-123.562
411,-122.871,21.247,21.247,20.439,-122.871,-119.302
412,-122.755,21.083,21.083,16.252,-122.755,-101.167
413,-122.272,20.831,20.831,14.861,-122.272,-149.760


In [107]:
df_23043.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location-long  415 non-null    float64
 1   location-lat   415 non-null    float64
 2   argos:lat1     415 non-null    float64
 3   argos:lat2     415 non-null    float64
 4   argos:lon1     415 non-null    float64
 5   argos:lon2     415 non-null    float64
dtypes: float64(6)
memory usage: 19.6 KB


In [108]:
n = len(df_23043)
train_df = df_23043[0:int(n*0.7)]
val_df = df_23043[int(n*0.7):int(n*0.9)]
test_df = df_23043[int(n*0.9):]

num_features = df_23043.shape[1]


In [109]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
train_df = tf.keras.utils.timeseries_dataset_from_array(
    train_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [110]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
test_df = tf.keras.utils.timeseries_dataset_from_array(
    test_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [111]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
val_df = tf.keras.utils.timeseries_dataset_from_array(
    val_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [112]:
from tensorflow.data import Dataset

In [113]:
def mapdataset(elem):
    X,y = elem[:,:-1,:],elem[:,-1:,0:2]
    X.set_shape([None, n_points, None])
    y.set_shape([None, window_length - n_points, None])
    
    return X,y

In [114]:
train = train_df.map(mapdataset)

In [115]:
test = test_df.map(mapdataset)

In [116]:
val = val_df.map(mapdataset)

In [62]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=2)
])

In [66]:
MAX_EPOCHS = 300

def compile_and_fit(lstm_model, train, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                    patience=patience,
                                                    mode='min')
    
    compile = lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
    
    
    history = lstm_model.fit(train, epochs=MAX_EPOCHS,
                      validation_data=val,
                      callbacks=[early_stopping])
    return history

In [68]:
history = compile_and_fit(lstm_model, train)

Epoch 1/300
9/9 [==============================] - 1s 52ms/step - loss: 2644.4973 - mean_absolute_error: 37.6571 - val_loss: 2518.4692 - val_mean_absolute_error: 37.0841
Epoch 2/300
9/9 [==============================] - 0s 17ms/step - loss: 2631.6367 - mean_absolute_error: 37.4887 - val_loss: 2506.2439 - val_mean_absolute_error: 36.9810
Epoch 3/300
9/9 [==============================] - 0s 18ms/step - loss: 2618.8604 - mean_absolute_error: 37.3248 - val_loss: 2494.0862 - val_mean_absolute_error: 36.8829
Epoch 4/300
9/9 [==============================] - 0s 17ms/step - loss: 2606.1616 - mean_absolute_error: 37.1636 - val_loss: 2481.9961 - val_mean_absolute_error: 36.7915
Epoch 5/300
9/9 [==============================] - 0s 17ms/step - loss: 2593.5413 - mean_absolute_error: 37.0057 - val_loss: 2469.9731 - val_mean_absolute_error: 36.7044
Epoch 6/300
9/9 [==============================] - 0s 18ms/step - loss: 2580.9978 - mean_absolute_error: 36.8511 - val_loss: 2458.0146 - val_mean_abso

Epoch 49/300
9/9 [==============================] - 0s 16ms/step - loss: 2091.4880 - mean_absolute_error: 33.1217 - val_loss: 1985.6431 - val_mean_absolute_error: 33.1721
Epoch 50/300
9/9 [==============================] - 0s 17ms/step - loss: 2081.0295 - mean_absolute_error: 33.0412 - val_loss: 1975.4983 - val_mean_absolute_error: 33.0913
Epoch 51/300
9/9 [==============================] - 0s 17ms/step - loss: 2070.6094 - mean_absolute_error: 32.9607 - val_loss: 1965.3914 - val_mean_absolute_error: 33.0106
Epoch 52/300
9/9 [==============================] - 0s 17ms/step - loss: 2060.2278 - mean_absolute_error: 32.8802 - val_loss: 1955.3210 - val_mean_absolute_error: 32.9299
Epoch 53/300
9/9 [==============================] - 0s 17ms/step - loss: 2049.8843 - mean_absolute_error: 32.7997 - val_loss: 1945.2889 - val_mean_absolute_error: 32.8493
Epoch 54/300
9/9 [==============================] - 0s 17ms/step - loss: 2039.5790 - mean_absolute_error: 32.7194 - val_loss: 1935.2936 - val_mea

Epoch 97/300
9/9 [==============================] - 0s 18ms/step - loss: 1631.0197 - mean_absolute_error: 29.3331 - val_loss: 1539.5717 - val_mean_absolute_error: 29.3830
Epoch 98/300
9/9 [==============================] - 0s 17ms/step - loss: 1622.2893 - mean_absolute_error: 29.2563 - val_loss: 1531.1312 - val_mean_absolute_error: 29.3062
Epoch 99/300
9/9 [==============================] - 0s 17ms/step - loss: 1613.5929 - mean_absolute_error: 29.1795 - val_loss: 1522.7242 - val_mean_absolute_error: 29.2295
Epoch 100/300
9/9 [==============================] - 0s 17ms/step - loss: 1604.9298 - mean_absolute_error: 29.1028 - val_loss: 1514.3499 - val_mean_absolute_error: 29.1528
Epoch 101/300
9/9 [==============================] - 0s 19ms/step - loss: 1596.3004 - mean_absolute_error: 29.0262 - val_loss: 1506.0092 - val_mean_absolute_error: 29.0763
Epoch 102/300
9/9 [==============================] - 0s 16ms/step - loss: 1587.7043 - mean_absolute_error: 28.9497 - val_loss: 1497.7009 - val_

Epoch 145/300
9/9 [==============================] - 0s 17ms/step - loss: 1208.6133 - mean_absolute_error: 25.3373 - val_loss: 1131.8328 - val_mean_absolute_error: 25.3865
Epoch 146/300
9/9 [==============================] - 0s 17ms/step - loss: 1200.9479 - mean_absolute_error: 25.2588 - val_loss: 1124.4678 - val_mean_absolute_error: 25.3080
Epoch 147/300
9/9 [==============================] - 0s 16ms/step - loss: 1193.3241 - mean_absolute_error: 25.1804 - val_loss: 1117.1427 - val_mean_absolute_error: 25.2297
Epoch 148/300
9/9 [==============================] - 0s 16ms/step - loss: 1185.7406 - mean_absolute_error: 25.1022 - val_loss: 1109.8579 - val_mean_absolute_error: 25.1516
Epoch 149/300
9/9 [==============================] - 0s 16ms/step - loss: 1178.1981 - mean_absolute_error: 25.0241 - val_loss: 1102.6128 - val_mean_absolute_error: 25.0736
Epoch 150/300
9/9 [==============================] - 0s 16ms/step - loss: 1170.6954 - mean_absolute_error: 24.9462 - val_loss: 1095.4070 - v

Epoch 193/300
9/9 [==============================] - 0s 17ms/step - loss: 881.8345 - mean_absolute_error: 21.7259 - val_loss: 818.7091 - val_mean_absolute_error: 21.7789
Epoch 194/300
9/9 [==============================] - 0s 17ms/step - loss: 875.8333 - mean_absolute_error: 21.6537 - val_loss: 812.9791 - val_mean_absolute_error: 21.7067
Epoch 195/300
9/9 [==============================] - 0s 16ms/step - loss: 869.8625 - mean_absolute_error: 21.5816 - val_loss: 807.2792 - val_mean_absolute_error: 21.6347
Epoch 196/300
9/9 [==============================] - 0s 16ms/step - loss: 863.9216 - mean_absolute_error: 21.5096 - val_loss: 801.6084 - val_mean_absolute_error: 21.5628
Epoch 197/300
9/9 [==============================] - 0s 16ms/step - loss: 858.0106 - mean_absolute_error: 21.4377 - val_loss: 795.9674 - val_mean_absolute_error: 21.4909
Epoch 198/300
9/9 [==============================] - 0s 17ms/step - loss: 852.1294 - mean_absolute_error: 21.3659 - val_loss: 790.3553 - val_mean_abso

9/9 [==============================] - 0s 17ms/step - loss: 625.8890 - mean_absolute_error: 18.3822 - val_loss: 575.2798 - val_mean_absolute_error: 18.4384
Epoch 242/300
9/9 [==============================] - 0s 17ms/step - loss: 621.2145 - mean_absolute_error: 18.3152 - val_loss: 570.8557 - val_mean_absolute_error: 18.3714
Epoch 243/300
9/9 [==============================] - 0s 17ms/step - loss: 616.5650 - mean_absolute_error: 18.2482 - val_loss: 566.4557 - val_mean_absolute_error: 18.3045
Epoch 244/300
9/9 [==============================] - 0s 17ms/step - loss: 611.9407 - mean_absolute_error: 18.1814 - val_loss: 562.0809 - val_mean_absolute_error: 18.2377
Epoch 245/300
9/9 [==============================] - 0s 17ms/step - loss: 607.3416 - mean_absolute_error: 18.1146 - val_loss: 557.7308 - val_mean_absolute_error: 18.1711
Epoch 246/300
9/9 [==============================] - 0s 17ms/step - loss: 602.7673 - mean_absolute_error: 18.0480 - val_loss: 553.4055 - val_mean_absolute_error: 18

9/9 [==============================] - 0s 17ms/step - loss: 428.6075 - mean_absolute_error: 15.2821 - val_loss: 389.5391 - val_mean_absolute_error: 15.3414
Epoch 290/300
9/9 [==============================] - 0s 17ms/step - loss: 425.0556 - mean_absolute_error: 15.2201 - val_loss: 386.2175 - val_mean_absolute_error: 15.2795
Epoch 291/300
9/9 [==============================] - 0s 17ms/step - loss: 421.5250 - mean_absolute_error: 15.1582 - val_loss: 382.9165 - val_mean_absolute_error: 15.2177
Epoch 292/300
9/9 [==============================] - 0s 17ms/step - loss: 418.0155 - mean_absolute_error: 15.0965 - val_loss: 379.6364 - val_mean_absolute_error: 15.1560
Epoch 293/300
9/9 [==============================] - 0s 17ms/step - loss: 414.5277 - mean_absolute_error: 15.0348 - val_loss: 376.3774 - val_mean_absolute_error: 15.0944
Epoch 294/300
9/9 [==============================] - 0s 16ms/step - loss: 411.0610 - mean_absolute_error: 14.9732 - val_loss: 373.1396 - val_mean_absolute_error: 15